# Domain Description

The model should have 50x50 cells, each 1000 m in x and in y. The porosity is 0.10, specific yield is 0.10, and storage coefficient is 0.0001. There are three layers. The medium is homogeneous within each layer. The K of the top and bottom layers is 10 m/day in all three principal directions. K of the middle layer is the same as the lower layer in the leftmost 20 columns, but it is 0.0001 m/day in the z direction in the remaining columns. The bottom of the domain is topographically flat and the bottom layer is 40 m thick. The middle layer is 5 m thick and is also flat. The top layer elevation is provided in an Excel file below. The top left and bottom left corners of the domain are 'rounded' by bedrock. Specifically, in the top, there is a triangle of no flow cells (added under BCs) extending from row 45, column 1 to row 50, column 6, inclusive, comprising a total of 21 no flow cells. There is a symmetric no flow region in the top left corner. The middle layer has similar regions extending from row 43, column 1 to row 50, column 8. The bottom layer: row 41, column 1 to row 50, column 10.

The right boundary in all of the layers has a constant head of 70 m relative to the datum, which is located at the bottom of the domain. All other boundaries are no flow.

Recharge occurs at a rate of 4E-5 m/day in the leftmost 15 columns and is zero elsewhere.

A stream extends from the left to the right boundary in row 26. The stream width, length, and thickness are 1.0. No flow is entering the stream (from tributaries).

The K of the streambed is 1000 m/day The streambed elevation is 1.0 m below ground surface and the stage is 0.5 m.

The stream is a 'weak sink' with a strength of 0.5, meaning that half of the particles that enter a stream cell are captured by the stream. (This is set under MODPATH/Particle Options.).

ET is zero in the left half of the domain. ET is 1E-5 m/day in the right half of the domain. ET occurs at a rate of 5E-5 m/day in a riparian area that extends from the left boundary to the right boundary and occupies rows 23 to 29, inclusive. The extinction depth is 10 m everywhere.

There is a well that is used for water supply by the local community, which is completed in the bottom layer at row 21 and column 38. It is pumped at a rate of 1500 m3/day.

The description above defines the system before a proposed new agricultural project is realized. The ag field is proposed to cover a 2000 m by 2000 m area; 1/8th of the area will be irrigated agriculture at any time (the field area is multiplied by 0.125). The rectangular irrigated fields extend between rows 21 and 22 (inclusive) and columns 19 and 20 (inclusive).

ET for the crop is zero - it is accounted for in the calculated recharge beneath the field. The recharge rate is assumed to be 20% of the water demand of the crop, representing intentional excess irrigation to avoid soil salinization. The water uses of wheat, pistachios, and cotton on a daily basis are: 0.004; 0.006; and 0.008 m/day. This leads to recharge rates of (e.g. 0.004 * 0.125 * 0.2 = 0.0001): 0.0001, 0.00015, and 0.0002 m/day for these crops, respectively.

Water is provided for irrigation from a well that is completed in the top layer at row 12 and column 14. The pumping rate is equal to the crop water demand plus 20% for excess irrigation plus 30% for irrigation inefficiency. For wheat, the pumping rate is 0.004 *0.125 * (1.0+0.2+0.3) * 2000 * 2000 = 3000 m3/day). The resulting pumping rates are: 3000; 4500; and 6000 m3/day for wheat, pistachios, and cotton, respectively.

Add two ‘monitoring wells’ to monitor the transient conditions. Both should be in layer one, one at [25000, 25000] and the other at [12500, 12500].

For simplicity, we will consider a year to be comprised of 12 30-day months.

# Scenarios

## Scenario 1: Pre Development model, no seasonality

Build the base model as described above without the proposed agricultural activity.
 * Run the model as steady state with no pumping from the town's well.
 * Calculate the flux from the stream to the groundwater
 * Also show a reverse particle track map to identify the source of the water to the stream.
 * Finally, report the water level at the monitoring wells and at the town's well (even though it isn't pumping for this scenario).

# Import

In [3]:
import flopy
import numpy as np
import matplotlib as mp
import os
import flopy.utils.binaryfile as bf
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import csv
%matplotlib inline

flopy is installed in /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/flopy


# Inputs

In [97]:
nrow=50
ncol=50
nlay=3
dx=ncol/dx
dy=nrow/dy
Lx=1000
Ly=1000
ztop=np.genfromtxt('base_model_elevs.csv',delimiter=',',dtype=float)
ztop[0,0]=125
zbot=np.zeros((nlay,nrow,ncol))
zbot[1,:,:]=45
zbot[2,:,:]=40
nper=1
steady=[True]
hka=10
vka=10
kz=np.zeros((nlay,nrow,ncol))
kz[:,:,:]=vka
kz[1,:,20:]=1e-4
kh=np.zeros((nlay,nrow,ncol))
kh[:,:,:]=hka
n=0.1
Sy=0.1
Ss=1e-4
ibound=np.ones((nlay,nrow,ncol))
ibound[:,0,:49]=-1
ibound[:,:,0]=-1
ibound[:,49,:49]=-1
ibound[:,1,:5]=-1
ibound[:,2,:4]=-1
ibound[:,3,:3]=-1
ibound[:,4,:2]=-1
ibound[:,48,:5]=-1
ibound[:,47,:4]=-1
ibound[:,46,:3]=-1
ibound[:,45,:2]=-1

ibound[1:,1,5:7]=-1
ibound[1:,2,4:6]=-1
ibound[1:,3,3:5]=-1
ibound[1:,4,2:4]=-1
ibound[1:,5,1:3]=-1
ibound[1:,6,1]=-1
ibound[1:,48,5:7]=-1
ibound[1:,47,4:6]=-1
ibound[1:,46,3:5]=-1
ibound[1:,45,2:4]=-1
ibound[1:,44,1:3]=-1
ibound[1:,43,1]=-1

ibound[2,1,7:9]=-1
ibound[2,2,6:8]=-1
ibound[2,3,5:7]=-1
ibound[2,4,4:6]=-1
ibound[2,5,3:5]=-1
ibound[2,6,2:4]=-1
ibound[2,7,1:3]=-1
ibound[2,8,1]=-1

ibound[2,48,7:9]=-1
ibound[2,47,6:8]=-1
ibound[2,46,5:7]=-1
ibound[2,45,4:6]=-1
ibound[2,44,3:5]=-1
ibound[2,43,2:4]=-1
ibound[2,42,1:3]=-1
ibound[2,41,1]=-1

head=np.zeros((nlay,nrow,ncol),dtype=np.float32)
head[:,:,49]=70

rech=np.zeros((nlay,nrow,ncol))
rech[:,:,:16]=4e-5

riv_sp1=[]
k_rivbott=1000 
sed_thick=1 
cond=k_rivbott*(dy)*(dx)/(sed_thick) 
r_stage=0.5 
r_bott=49 
for i in range(nrow):
    riv_sp1.append([0,26,i,r_stage,cond,r_bott])
riv_spd={0:riv_sp1}